# Basic setup for fetching data from the facebook api

### Remember to add an active access token below and adjust the parameters and fields if necessary

In [10]:
# must be specified to be able to use methods of package

import sys
sys.path.append('../src') 
# import packages
from political_ads.helper import *
import pandas as pd
from political_ads.api_request import get_json_response



In [11]:
# insert your access token here (expires every 2 hours)
access_token = "EAADqFZC76BDMBAKivIHmFy5WyaJPfYxLAejvA4Oa3XqA7Ip6AsuGyuxoV9x1uQqc4115kZCoOYZBajckWtfVtsucu9VMHeInCH2A4v6TTQWLQnMI2YvhfnSas9EXUogIf88thIDjPja2Xh8mOrVYiqi7YTeUddHhyUCqZAnQuZCHk9P6IsxRg7ZCCdCNIZBjZC7cRrT5goo06ZALs10Kb4LI3fMqCzNPWNiiXN3jP5xy0qLi6mKyQZBAF0ujWwZB4z9jYoZD"
# get_new_access_token: https://developers.facebook.com/tools/explorer/


# PARAMETERS NECESSARY
ad_type = "POLITICAL_AND_ISSUE_ADS"
ad_reached_countries = ['US'] # Facebook delivered the ads in these countries. Provided as ISO country codes.
search_terms = "climate change" # The terms to search for in your query. We treat a blank space as a logical AND and search for both terms and no other operators. The limit of your string is 100 characters or less.
limit = 1700
start_date = "2020-08-00"
end_date = "2020-11-03"


# FIELDS to specify your results
fields = ["id", "page_name", "ad_creative_body", "impressions", "delivery_by_region", "demographic_distribution", "ad_delivery_start_time", "bylines", "spend"]

# see here for more parameters and fields: https://www.facebook.com/ads/library/api/?source=archive-landing-page 

input_url = f"https://graph.facebook.com/v12.0/ads_archive?search_terms={search_terms}&ad_type={ad_type}&ad_reached_countries={ad_reached_countries}&fields={fields}&limit={limit}&ad_delivery_date_min={start_date}&ad_delivery_date_max={end_date}"

response = get_json_response(input_url, access_token)

data = pd.DataFrame(response["data"])


In [25]:
import time 

#https://developers.facebook.com/docs/graph-api/results

iterations = 10
sleep_time_sec = 10

counter = 0

for i in range(10):

    paging = response["paging"]["next"]
    next_response = get_json_response(paging, access_token)

    new_page = pd.DataFrame(next_response['data'])
    data = data.append(new_page, ignore_index=True)
    time.sleep(10)
    counter+=1
    print(counter)




1
2
3
4
5
6
7
8
9
10


In [26]:
#Write to CSV
#data.to_csv('test.csv')

In [12]:
#Cleaning the Data // Dropping missing values
data.dropna(inplace=True)


In [85]:
# data.drop(['age_distribution', 'gender'], axis=1)

#data['age_distribution'] = data['demographic_distribution'].apply(lambda x: get_age_distribution(x, all_ages=True))

demographic_data = data[['page_name', 'demographic_distribution']].reset_index().drop(columns='index')

data['demographic_distribution']

# data['impressions'] = data.impressions.apply(lambda x: transform_range(x))

demographic_data['gender (M)'] = data['demographic_distribution'].apply(lambda x: get_gender_distribution(x))


#method in the making
def demographic_overview(df, colname='demographic_distribution'):
    for i in range(len(df)):
        for j in range(len(df[colname][i])):
            df[colname][i][j]


#test run
df = demographic_data
colname = 'demographic_distribution'

#Creating the Columns
age_dist_lst = ['13-17', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
for i in age_dist_lst:
     df[i] = 0

#filling in the data in the columns
for i in range(len(df)):
    for j in range(len(df[colname][i])):
        demo_group_per = df[colname][i][j]
        df[demo_group_per['age']].iloc[i] += float(demo_group_per['percentage'])


demographic_data.groupby('page_name').mean()


/home/gustavgyrst/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,gender (M),13-17,18-24,25-34,35-44,45-54,55-64,65+
page_name,,,,,,,,
100 Percent Campaign,0.407408,0.000000,0.330919,0.501943,0.167138,0.000000,0.000000,0.000000
314 Action,0.575471,0.000000,0.083886,0.216309,0.183721,0.137100,0.165858,0.213126
350 New Hampshire Action,0.217004,0.000000,0.026313,0.086115,0.091303,0.170385,0.307597,0.318289
A. Donald McEachin,0.433457,0.000000,0.183981,0.379948,0.181622,0.113661,0.097938,0.042852
Action for the Climate Emergency,0.245373,0.317789,0.329307,0.292932,0.059973,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
World War Zero,0.296404,0.000000,0.253215,0.173197,0.067535,0.099208,0.178763,0.228083
YES on 2A - Climate Action Now,0.332217,0.000000,0.099135,0.349947,0.230191,0.129479,0.095927,0.095321
YES on Measure 26-218,0.577392,0.000000,0.153012,0.334747,0.214571,0.122488,0.087512,0.087669


In [ ]:


#df.loc[df['B'].isin(['one','three'])])

# Two ways to attack this filtering of the data:
# data.loc[data['page_name'].isin(['Boost the News'])]
# data.loc[data['page_name'] == 'Boost the News']

# data['page_name'].str.contains('Boost', 'Climate').any()

# data['page_name'].filter(regex='Boost')

# @USCongressmanFilemonVela etc.
set_of_pol_pgnames = {'USCongressmanFilemonVela', 'reppaultonko', 'SenatorThomTillis', 'DonaldNorcrossNJ', '153423912663', 'repalgreen', 'RepDonBeyer', 'RepRogerWilliams', 'CongressmanSethMoulton', 'replizcheney', 'RepRoybalAllard', 'mitchmcconnell', 'SenatorRandPaul', 'RepSteveScalise', 'RepMoBrooks', 'CongressmanAlexMooney', 'RepJackBergman', 'RepWesterman', 'RepMcEachin', 'CongresswomanAnnieKuster', 'RepFrankLucas', 'CongressmanTedDeutch', 'GwenSMoore', 'RepTedLieu', 'RepGregMurphy', 'senatordebfischer', 'RepBobGibbs', 'AndyHarrisMD', 'WhipHoyer', 'CongressmanMichaelF.Q.SanNicolas', 'congressmangkbutterfield', 'SenatorTimScott', 'RepJohnCurtis', 'CongressmanNadler', '50375006903', 'RepBarbaraLee', 'jeffmerkley', 'JoeManchinIII', 'RepBost', 'CongressmanBuchanan', 'RepJeffDuncan', 'Congressman.Keating', 'RepSewell', 'RepRickCrawford', 'RepChrisStewart', 'zoelofgren', 'CongressmanWarrenDavidson', 'drnealdunnfl2', 'RepTomSuozzi', 'SenatorLankford', 'RepCheri', 'RepKinzinger', 'marshablackburn', 'SenBrianSchatz', 'RepDonBacon', 'SenatorHassan', 'senshelley', 'senatorcantwell', 'BlaineLuetkemeyer', '113303673339', 'RepJohnLarson', 'senatortester', 'RepRouzer', 'RobertAderholt', 'reptrey', 'jameseclyburn', 'RepRichHudson', 'RickScottSenOffice', 'SenGaryPeters', 'repschrader', 'RepKweisiMfume', 'CongressmanSteveCohen', 'senatortammybaldwin', 'RepAdamSchiff', 'RepBobbyScott', 'RepEliseStefanik', 'senatorelizabethwarren', 'CongressmanMattCartwright', 'RepAmiBera', 'CongressmanKevinMcCarthy', 'RepAnthonyBrown', 'repgaramendi', 'CongressmanDavidCicilline', 'RepJimBanks', 'JodeyArrington', 'repgracemeng', 'bill.posey15', 'senatorchriscoons', 'RepTedBudd', 'RepBrianHiggins', 'RepMarkTakano', 'CongressmanBennieGThompson', 'emanuelcleaverii', 'RepMarkDeSaulnier', 'RepHuffman', 'SenatorTomCotton', 'repscottperry', 'RepresentativeValDemings', 'senatorjimrisch', 'ScottDesJarlaisTN04', 'RepLarryBucshon', 'repchuck', 'RepWalberg', 'senbennetco', 'RepLoisFrankel', 'reptomrice', 'CongressmanClayHiggins', 'SenatorJohnHoeven', 'RepBonnie', 'RepHuizenga', 'CongressmanMikeDRogers', 'RepJoyceBeatty', 'SenatorSasse', 'CongressmanMarcVeasey', 'DianaDeGette', 'chrismurphyct', 'gregorymeeksny05', 'CongressmanDKDavis', 'CongresswomanRosaDeLauro', 'senatorbencardin', 'SenatorMikeRounds', 'RepAustinScott', 'mcmorrisrodgers', '152569121550', 'RepJuliaBrownley', 'USSenTinaSmith', 'repmarkpocan', 'timryan', 'herrerabeutler', 'sen.johncornyn', 'repstephenlynch', 'RepJoshG', 'RepJuanVargas', 'CongressmanGuthrie', 'repfrankpallone', '81058818750', 'tomcarper', 'repgosar', 'senjoniernst', 'CongressmanBillFoster', 'repcardenas', 'SenatorKaine', 'RepJimMcGovern', 'RepConorLamb', 'RepAdamSmith', 'RepDavidKustoff', 'RepDebbieLesko', 'RepKathleenRice', 'johnbarrasso', 'SenJackReed', 'RepJimmyPanetta', 'RepSteveChabot', 'reprobinkelly', 'pascrell', 'CongressmanKevinCramer', 'CongressmanGarretGraves', 'JohnKennedyLouisiana', 'senatorfeinstein', 'RepMullin', 'RepRubenGallego', 'RepScottPeters', 'joecourtney', 'RepSmucker', 'TomColeOK04', 'CongresswomanLindaSanchez', 'CongressmanDougLamborn', 'RepMikeJohnson', 'SenatorRichardBurr', 'repbrianmast', 'repjimjordan', 'RepMikeTurner', 'DonaldPayneJr', 'SenatorPatrickLeahy', 'RepTomReed', 'CongressmanBoyle', 'SenatorCortezMasto', 'ChelliePingree', 'RepRoKhanna', 'senatormikelee', 'CongressmanHalRogers', 'SenatorMarcoRubio', 'jerrymoran', 'chrisvanhollen', 'RepDelBene', 'CongressmanJimHimes', 'RepRobWittman', 'CongressmanRickAllen', 'jiminhofe', 'RepEspaillat', 'RepDebbieDingell', 'RepBradWenstrup', 'RepKarenBass', 'TXRandy14', 'RepFredUpton', 'SenatorTedCruz', 'RepresentativeMarcyKaptur', 'RepMikeGallagher', 'RepDonYoung', '395759603917487', 'susancollins', 'RepBillJohnson', '118514606128', 'RepRonEstes', 'RepSeanMaloney', 'RepMcKinley', 'SenatorAngusSKingJr', 'senronjohnson', 'repdavidschweikert', 'RepMikeThompson', 'RepMoolenaar', 'RepAndyBiggs', 'RepStephMurphy', 'sanfordbishop', 'RepSarbanes', 'CongressmanGaryPalmer', 'RepSteveWomack', 'RepLeeZeldin', 'RepDanKildee', 'JoeWilson', '81125319109', 'congressmancomer', '96007744606', 'GusBilirakis', '115356957005', 'RepDwightEvans', 'RepPerlmutter', 'RepJackieWalorski', 'CongressmanBradSchneider', 'CongressmanFredKeller', 'aumuaamata', 'CongressmanJodyHice', 'reprichardneal', 'replahood', 'SenatorDurbin', 'RepGrothman', 'RepLouCorrea', 'SenatorWhitehouse', 'RepLaMalfa', 'CongressmanMcHenry', 'RepVirginiaFoxx', 'CongressmanGerryConnolly', 'SenDanSullivan', 'RepAndyBarr', 'repbrianfitzpatrick', 'mdiazbalart', 'SenatorBlunt', 'CongresswomanTitus', 'stevenpalazzo', 'USSenatorLindseyGraham', 'RepDrewFerguson', 'mikecrapo', 'SenatorStabenow', 'DutchRupp', 'lloyddoggett', 'CongressmanEricSwalwell', 'judgecarter', 'RepJimCosta', 'RepAnnaEshoo', 'RepJackyRosen', 'CongressmanRaulRuizMD', 'senatorhirono', 'RepThomasMassie', 'congresswomanbonamici', 'SenLisaMurkowski', 'RepWilson', 'RepNormaTorres', 'RepJohnKatko', 'RepGraceNapolitano', 'JohnBoozman', 'RepHakeemJeffries', 'RepJudyChu', 'RepDaveJoyce', 'MartinHeinrich', '191056827594903', 'reploudermilk', 'CongressmanMattGaetz', 'jefffortenberry', 'RepRickLarsen', 'SenBlumenthal', 'RepAnnWagner', 'michaeltmccaul', 'RepRodneyDavis', 'RichardShelby', 'RepChrisSmith', 'doris.matsui', 'RepRutherfordFL', 'repsaludcarbajal', 'PeterWelch', 'CongressmanAndreCarson', 'NancyPelosi', 'SenDuckworth', 'CongressmanGT', 'congressmanraja', 'CongresswomanNorton', 'senatormenendez', 'JimCooper', 'MaxineWaters', 'SteveDainesMT', 'USCongressmanVicenteGonzalez', 'RepCarolynMaloney', 'CongressmanDarrenSoto', 'senschumer', 'repstaceyplaskett', 'CongresswomanAdams', 'Rep.Billy.Long', 'congressmanbuddycarter', 'SenatorShaheen', 'USRepKathyCastor', 'boblatta', 'CongresswomanSinema', 'RepJennifferGonzalezColon', 'repbettymccollum', 'repyvettedclarke', 'RepCharlieCrist', 'usrepmikedoyle', 'senatortoomey', 'RepJayapal', '326420614138023', 'RepLowenthal', 'SenatorSherrodBrown', 'jerrymcnerney', 'repjasonsmith', 'CongresswomanEBJtx30', 'CongresswomanSheilaJacksonLee', '70063393423', '63158229861', 'RepWebster', '214258646163', 'JackieSpeier', 'kevinbrady', 'senrobportman', 'Rep.BluntRochester', 'RepBrianBabin', 'repraskin', 'reptomemmer', 'repdanbishop', 'repmikequigley', 'MarkRWarner', 'EdJMarkey', 'senatorsanders', 'repohalleran', 'grassley', 'RepRalphNorman', 'RepDWS', 'RepMorganGriffith', 'MarkAmodeiNV2', 'RepJimmyGomez', 'Rep.Grijalva', 'repronkind', 'janschakowsky', 'SenKirstenGillibrand', 'repkenbuck', 'reppeteaguilar', 'RepDavidEPrice', 'SenatorToddYoung', 'CongressmanJimLangevin', 'RepFrenchHill', 'CongresswomanClark', 'SenatorBobCasey', 'Congresswoman.Hartzler', 'derek.kilmer', 'reptrentkelly', 'SenMarkKelly', 'congressmanbobbyrush', 'CongresswomanBarragan', 'RepKayGranger', 'michaelcburgess', 'RepPeterDeFazio', 'SenatorWicker', 'RepMikeGarcia', 'RepAlLawsonJr', 'RepNewhouse', '8037068318'}

set_of_pol_full_names = {'Brendan F. Boyle', 'Andy Kim', 'Lauren Underwood', 'Thomas P. Tiffany', 'Rand Paul', 'Mike Rogers', 'Salud O. Carbajal', 'Gary C. Peters', 'John Katko', 'Greg Pence', 'Eddie Bernice Johnson', 'Jim Costa', 'Aumua Amata Coleman Radewagen', 'Suzanne Bonamici', 'Tony Gonzales', 'Raja Krishnamoorthi', 'David B. McKinley', 'Stephanie I. Bice', 'Victoria Spartz', 'A. Drew Ferguson IV', 'Mary E. Miller', 'Lucille Roybal-Allard', 'Kirsten E. Gillibrand', 'Jeff Fortenberry', 'Darren Soto', 'John B. Larson', 'Troy Balderson', 'James Comer', 'Ken Buck', 'Russ Fulcher', 'Eric A. "Rick" Crawford', 'Brian Schatz', 'Doug Lamborn', 'Henry C. "Hank" Johnson, Jr.', 'Tom Emmer', 'Sam Graves', 'Alexandria Ocasio-Cortez', 'Richard Burr', 'Patty Murray', 'Ed Perlmutter', 'Charlie Crist', 'Kevin Brady', 'Lisa C. McClain', 'Cori Bush', 'Bill Cassidy', 'Mike Garcia', 'Ben Sasse', 'Tina Smith', 'Robert B. Aderholt', 'Mark DeSaulnier', 'Nicole Malliotakis', 'Al Green', 'Mariannette Miller-Meeks', 'Thomas R. Suozzi', 'Christopher H. Smith', 'Debbie Wasserman Schultz', 'Trey Hollingsworth', 'Mike Levin', 'Troy E. Nehls', 'Zoe Lofgren', 'Debbie Lesko', 'Mo Brooks', 'Matthew M. Rosendale', 'Mark E. Amodei', 'Rick W. Allen', 'Bill Hagerty', 'Donald Norcross', 'Marie Newman', 'Chris Van Hollen', 'Robert J. Wittman', 'Grace Meng', 'Nikema Williams', 'Dean Phillips', 'Pat Fallon', 'Eric Swalwell', 'Earl L. "Buddy" Carter', 'Kurt Schrader', 'Dan Bishop', 'Sherrod Brown', 'Ted Lieu', 'Jason Crow', 'Judy Chu', 'Cathy McMorris Rodgers', 'Anthony Gonzalez', 'Kaiali’i Kahele', 'Cynthia Axne', 'Jody B. Hice', 'Jack Bergman', 'Nancy Mace', 'Kevin Hern', 'Richard Blumenthal', 'Kathy E. Manning', 'David Kustoff', 'Greg Stanton', 'Richard C. Shelby', 'Michael T. McCaul', 'Tim Walberg', 'David N. Cicilline', 'Mark R. Warner', 'Paul Tonko', 'Val Butler Demings', 'Mark E. Green', 'Lois Frankel', 'Steve Daines', 'Tracey Mann', 'Guy Reschenthaler', 'Kat Cammack', 'Roy Blunt', 'André Carson', 'Alexander Mooney', 'Adrian Smith', 'Bob Gibbs', 'Pete Aguilar', 'Richard E. Neal', 'Ann Kirkpatrick', 'Fred Keller', 'Scott H. Peters', 'Andy Barr', 'Mark Kelly', 'Cindy Hyde-Smith', 'Joaquin Castro', 'Joe Wilson', 'Jack Reed', 'Lance Gooden', 'Thomas Massie', 'David Schweikert', 'Bill Johnson', 'John A. Yarmuth', 'Scott Fitzgerald', 'Roger Williams', 'Andy Biggs', 'Tom Malinowski', 'Joni Ernst', 'Angus S. King, Jr.', 'Patrick J. Leahy', 'C. A. Dutch Ruppersberger', 'John R. Moolenaar', 'Virginia Foxx', 'Tom Rice', 'John Thune', 'Barry Moore', 'Devin Nunes', 'Rashida Tlaib', 'Marc A. Veasey', 'Joe Neguse', 'Madison Cawthorn', 'Mike Bost', 'Pramila Jayapal', 'Bob Good', 'Jesús G. "Chuy" García', 'Larry Bucshon', 'Jared Huffman', 'Chris Jacobs', 'Mary Gay Scanlon', 'James E. Risch', 'Nancy Pelosi', 'Gregory W. Meeks', 'John W. Hickenlooper', 'Rick Scott', 'Jay Obernolte', 'Charles J. "Chuck" Fleischmann', 'Katherine M. Clark', 'Lizzie Fletcher', 'Jared F. Golden', 'Derek Kilmer', 'Tim Scott', 'Richard J. Durbin', 'Jim Cooper', 'Ann Wagner', 'Mondaire Jones', 'Michael C. Burgess', 'Cheri Bustos', 'August Pfluger', 'Sean Patrick Maloney', 'Edward J. Markey', 'Bernard Sanders', 'Ron Estes', 'Amy Klobuchar', 'Haley M. Stevens', 'Hakeem S. Jeffries', 'Dan Crenshaw', 'Tim Burchett', 'Tim Ryan', 'Michael K. Simpson', 'Diana DeGette', 'Todd Young', 'Melanie A. Stansbury', 'Lee M. Zeldin', 'Filemon Vela', 'Elissa Slotkin', 'Michael F. Bennet', 'Terri Sewell', 'Tom Cotton', 'Andrew R. Garbarino', 'Andrew S. Clyde', 'Markwayne Mullin', 'John Rose', 'James R. Langevin', 'Josh Harder', 'Burgess Owens', 'Stephen F. Lynch', 'Jake LaTurner', 'Rick Larsen', 'Jon Ossoff', 'Matt Cartwright', 'Andy Harris', 'Blaine Luetkemeyer', 'Michelle Steel', 'Garret Graves', 'Mikie Sherrill', 'Mike Quigley', 'Gregory F. Murphy', 'Richard Hudson', 'Liz Cheney', 'John P. Sarbanes', 'Elizabeth Warren', 'G. K. Butterfield', 'Ami Bera', 'Brad R. Wenstrup', 'James A. Himes', 'Neal P. Dunn', 'Byron Donalds', 'John Kennedy', 'Chellie Pingree', 'Chris Stewart', 'Bill Pascrell, Jr.', 'Brett Guthrie', 'Marsha Blackburn', 'Louie Gohmert', 'Lisa Blunt Rochester', 'Jenniffer González-Colón', 'Ken Calvert', 'Bradley Scott Schneider', 'Dan Sullivan', 'Don Bacon', 'Sean Casten', 'Brian K. Fitzpatrick', 'Kathleen M. Rice', 'Susie Lee', 'Yvette D. Clarke', 'Nydia M. Velázquez', 'Julia Brownley', 'Ayanna Pressley', 'William R. Timmons IV', 'Shelley Moore Capito', 'Mike Lee', 'Jahana Hayes', 'Michael F. Q. San Nicolas', 'Maria Elvira Salazar', 'Brian Babin', 'Chris Pappas', 'Carolyn Bourdeaux', 'Rob Portman', 'Maxine Waters', 'Jaime Herrera Beutler', 'Dianne Feinstein', 'Tom Cole', 'Anthony Brown', 'Steve Chabot', 'Ben Ray Luján', 'Thom Tillis', 'Mike Johnson', 'Karen Bass', 'Thomas R. Carper', 'Beth Van Duyne', 'Barry Loudermilk', 'Gus M. Bilirakis', 'Martin Heinrich', 'Donald S. Beyer, Jr.', 'Jennifer Wexton', 'Pete Sessions', 'Teresa Leger Fernandez', 'Gerald E. Connolly', 'Mike Kelly', 'Marco Rubio', 'Elaine G. Luria', 'Ted Budd', 'Rodney Davis', 'Adam Kinzinger', 'Yvette Herrell', 'Doug LaMalfa', 'Lucy McBath', 'Adam Smith', 'Lisa Murkowski', 'H. Morgan Griffith', 'Brenda L. Lawrence', 'Patrick T. McHenry', 'Jason Smith', 'Michael Guest', 'Ron Kind', 'Kyrsten Sinema', 'Jake Auchincloss', 'Bill Posey', 'Ralph Norman', 'Robert C. "Bobby" Scott', 'Michael Cloud', 'Jackie Speier', 'Margaret Wood Hassan', 'Tommy Tuberville', 'Gregorio Kilili Camacho Sablan', 'Dwight Evans', 'Ed Case', 'Josh Gottheimer', 'Lori Trahan', 'Lloyd Smucker', 'Raúl M. Grijalva', 'Jacky Rosen', 'Ron Wyden', 'Jimmy Gomez', 'David E. Price', 'J. Hill', 'Joe Manchin, III', 'Gwen Moore', 'Tom O’Halleran', 'Bobby L. Rush', 'Catherine Cortez Masto', 'Kim Schrier', 'Roger F. Wicker', 'Joseph D. Morelle', 'Dusty Johnson', 'Harold Rogers', 'Peter A. DeFazio', 'Trent Kelly', 'Christopher A. Coons', 'Lauren Boebert', 'Maria Cantwell', 'W. Gregory Steube', 'Patrick J. Toomey', 'Donald M. Payne, Jr.', 'Benjamin L. Cardin', 'Henry Cuellar', 'Steny H. Hoyer', 'Jim Banks', 'Josh Hawley', 'Carol D. Miller', 'Kathy Castor', 'Frederica S. Wilson', 'Robin L. Kelly', 'Tammy Baldwin', 'Paul A. Gosar', 'Elise M. Stefanik', 'James E. Clyburn', 'Robert P. Casey, Jr.', 'Daniel Webster', 'Cory A. Booker', 'Debbie Stabenow', 'Matt Gaetz', 'Steven Horsford', 'Ben Cline', 'James R. Baird', 'David G. Valadao', 'Lindsey Graham', 'Stephanie N. Murphy', 'Van Taylor', 'Vicente Gonzalez', 'Claudia Tenney', 'Don Young', 'David Rouzer', 'Peter Meijer', 'Nanette Diaz Barragán', 'Randy K. Weber, Sr.', 'Austin Scott', 'Tim Kaine', 'Tony Cárdenas', 'Joe Courtney', 'Tom Reed', 'Kay Granger', 'Ron Johnson', 'Warren Davidson', 'John H. Rutherford', 'Albio Sires', 'Ashley Hinson', 'Raphael G. Warnock', 'Fred Upton', 'John Barrasso', 'Ronny Jackson', 'Charles E. Schumer', 'Troy A. Carter', 'Seth Moulton', 'Tom McClintock', 'J. Luis Correa', 'Angie Craig', 'Frank D. Lucas', 'Roger Marshall', 'Mitch McConnell', 'Jackie Walorski', 'Joyce Beatty', 'Lloyd Doggett', 'David J. Trone', 'Brad Sherman', 'John R. Carter', 'Jamaal Bowman', 'Steve Womack', 'Jerry L. Carl', 'Michelle Fischbach', 'Mark Pocan', 'Glenn Grothman', 'Raul Ruiz', 'Sylvia R. Garcia', 'Jim Jordan', 'Mitt Romney', 'Deb Fischer', 'Peter Welch', 'Darrell Issa', 'James P. McGovern', 'Rosa L. DeLauro', 'Janice D. Schakowsky', 'Dina Titus', 'Sharice Davids', 'Mario Diaz-Balart', 'Blake D. Moore', 'Daniel T. Kildee', 'Jerry McNerney', 'Ritchie Torres', 'Adriano Espaillat', 'Andy Levin', 'Betty McCollum', 'Frank Pallone, Jr.', 'Randy Feenstra', 'John R. Curtis', 'Ann Kuster', 'Jim Hagedorn', 'Robert Menendez', 'Cynthia M. Lummis', 'Bennie G. Thompson', 'Jon Tester', 'Marcy Kaptur', 'Alma S. Adams', 'Suzan K. DelBene', 'Brian J. Mast', 'Jodey C. Arrington', 'James M. Inhofe', 'Conor Lamb', 'Madeleine Dean', 'Marjorie Taylor Greene', 'Steve Scalise', 'Sheila Jackson Lee', 'Daniel Meuser', 'Sanford D. Bishop, Jr.', 'Jeanne Shaheen', 'Ruben Gallego', 'A. Donald McEachin', 'Bill Huizenga', 'Doris O. Matsui', 'Bill Foster', 'Gary J. Palmer', 'Stacey E. Plaskett', 'Darin LaHood', 'Abigail Davis Spanberger', 'John Garamendi', 'Adam B. Schiff', 'David P. Joyce', 'Al Lawson, Jr.', 'Chrissy Houlahan', 'Mike Crapo', 'Glenn Thompson', 'Frank J. Mrvan', 'Jamie Raskin', 'Dan Newhouse', 'John Joyce', 'Susan Wild', 'Carlos A. Gimenez', 'Alex Padilla', 'Eleanor Holmes Norton', 'Alan S. Lowenthal', 'John Hoeven', 'Susan M. Collins', 'Mike Thompson', 'Pete Stauber', 'John Cornyn', 'Scott Perry', 'David Scott', 'Robert E. Latta', 'Jeff Duncan', 'Michael Waltz', 'Jimmy Panetta', 'Jefferson Van Drew', 'Veronica Escobar', 'Ilhan Omar', 'Anna G. Eshoo', 'Scott DesJarlais', 'Christopher Murphy', 'Chuck Grassley', 'William R. Keating', 'Debbie Dingell', 'Vern Buchanan', 'Chip Roy', 'Kelly Armstrong', 'Kevin McCarthy', 'Ro Khanna', 'Cliff Bentz', 'Diana Harshbarger', 'Brian Higgins', 'Norma J. Torres', 'Jerry Moran', 'Bryan Steil', 'Mike Braun', 'Carolyn B. Maloney', 'Earl Blumenauer', 'Deborah K. Ross', 'Mazie K. Hirono', 'Young Kim', 'Sara Jacobs', 'Barbara Lee', 'Tammy Duckworth', 'Julia Letlow', 'John Boozman', 'Bruce Westerman', 'Grace F. Napolitano', 'Linda T. Sánchez', 'Mark Takano', 'Katie Porter', 'Theodore E. Deutch', 'Mike Gallagher', 'Michael R. Turner', 'Ted Cruz', 'Sheldon Whitehouse', 'Bonnie Watson Coleman', 'James Lankford', 'Colin Z. Allred', 'Kevin Cramer', 'C. Scott Franklin', 'Steven M. Palazzo', 'Antonio Delgado', 'Jeff Merkley', 'Emanuel Cleaver', 'Danny K. Davis', 'Kweisi Mfume', 'Juan Vargas', 'Marilyn Strickland', 'Steve Cohen', 'Clay Higgins', 'Vicky Hartzler', 'Michael F. Doyle', 'Billy Long', 'Jerrold Nadler', 'Mike Rounds'}

white_house = {'Joe Biden', 'Kamala Harris', 'Barack Obama', 'Donald J. Trump', 'Mike Pence'}

combined = white_house.union(set_of_pol_full_names)

#To check for substring
data.loc[data['page_name'].str.contains('Congressman')]

data.loc[data['page_name'].str.contains('Congresswoman')]

data.loc[data['page_name'].str.contains('Senator')]
#Results with this

data.loc[data['page_name'].str.contains('Governor')]
#Results with this

#pol_fbpage_names = get_pol_fbpage_names()

df_by_pol = data.loc[data['page_name'].isin(combined)]

df_by_pol.groupby('page_name').count()





In [31]:
#https://datascience.stackexchange.com/questions/9616/how-to-create-us-state-choropleth-map
#Convert the demographic data into a heat-map



In [3]:
# Scraper experiement
# https://github.com/kevinzg/facebook-scraper

from facebook_scraper import get_page_info
from facebook_scraper import *

a = get_page_info(account="tomcarper")

print(a)


{'likes': 17074}


In [4]:
import requests
import time
from random import randint
import re




In [17]:

#Test1

name = 'SenatorThomTillis'
url  = 'https://www.facebook.com/' + name

r = requests.get(url)

pg_content = str(r.content)
look_up_word1 = '<title id="pageTitle">'

a = pg_content.split(look_up_word1)
b = a[-1].split(" -")

pol_fb_page_name = b[0]





In [7]:
#test2

url = 'https://www.facebook.com/repschrader'
url2 = 'https://www.facebook.com/SenatorThomTillis'
url3 = 'https://www.facebook.com/RepSewell'
url4 = 'https://www.facebook.com/SenatorRandPaul'

r = requests.get(url4)

if r.status_code == 200:
    pg_content = str(r.content)
    look_up_word = 'og:title" content="', 

    a = pg_content.split(look_up_word)
    b = a[-1].split('')

    pol_fb_page_name = b[0]

    print(pol_fb_page_name)



TypeError: must be str or None, not tuple

In [42]:

#https://developers.facebook.com/docs/pages/searching/

test_url = "https://graph.facebook.com/pages/search?q=SenatorThomTillis&fields=id,name,location,link&access_token=EAADqFZC76BDMBAKP4bH4gzbN37Eltae8mSRZC2mbJeAtMyFBEi58x6UMRZCBCJ8bAbCryoat10ss2FdQfWRWMeKk4l9ZA94Eg7c7GdxbljeByFKU7hZC8tNInIlaliiJ0MhCuU3wGJeQ9uzPboaIXARWXQQDwWcso3CqDjJc3OB4KmFw7i0GRnx7FzrkRHPmGBO0RVuZC7zZCydeM6zenwZAhfZBBukqIytAZD"

requests.get(test_url)

<Response [400]>

In [3]:

list_of_pol_pgnames = ['SenatorThomTillis', 'USCongressmanFilemonVela', 'reppaultonko', 'DonaldNorcrossNJ', '153423912663', 'repalgreen', 'RepDonBeyer', 'RepRogerWilliams', 'CongressmanSethMoulton', 'replizcheney', 'RepRoybalAllard', 'mitchmcconnell', 'SenatorRandPaul', 'RepSteveScalise', 'RepMoBrooks', 'CongressmanAlexMooney', 'RepJackBergman', 'RepWesterman', 'RepMcEachin', 'CongresswomanAnnieKuster', 'RepFrankLucas', 'CongressmanTedDeutch', 'GwenSMoore', 'RepTedLieu', 'RepGregMurphy', 'senatordebfischer', 'RepBobGibbs', 'AndyHarrisMD', 'WhipHoyer', 'CongressmanMichaelF.Q.SanNicolas', 'congressmangkbutterfield', 'SenatorTimScott', 'RepJohnCurtis', 'CongressmanNadler', '50375006903', 'RepBarbaraLee', 'jeffmerkley', 'JoeManchinIII', 'RepBost', 'CongressmanBuchanan', 'RepJeffDuncan', 'Congressman.Keating', 'RepSewell', 'RepRickCrawford', 'RepChrisStewart', 'zoelofgren', 'CongressmanWarrenDavidson', 'drnealdunnfl2', 'RepTomSuozzi', 'SenatorLankford', 'RepCheri', 'RepKinzinger', 'marshablackburn', 'SenBrianSchatz', 'RepDonBacon', 'SenatorHassan', 'senshelley', 'senatorcantwell', 'BlaineLuetkemeyer', '113303673339', 'RepJohnLarson', 'senatortester', 'RepRouzer', 'RobertAderholt', 'reptrey', 'jameseclyburn', 'RepRichHudson', 'RickScottSenOffice', 'SenGaryPeters', 'repschrader', 'RepKweisiMfume', 'CongressmanSteveCohen', 'senatortammybaldwin', 'RepAdamSchiff', 'RepBobbyScott', 'RepEliseStefanik', 'senatorelizabethwarren', 'CongressmanMattCartwright', 'RepAmiBera', 'CongressmanKevinMcCarthy', 'RepAnthonyBrown', 'repgaramendi', 'CongressmanDavidCicilline', 'RepJimBanks', 'JodeyArrington', 'repgracemeng', 'bill.posey15', 'senatorchriscoons', 'RepTedBudd', 'RepBrianHiggins', 'RepMarkTakano', 'CongressmanBennieGThompson', 'emanuelcleaverii', 'RepMarkDeSaulnier', 'RepHuffman', 'SenatorTomCotton', 'repscottperry', 'RepresentativeValDemings', 'senatorjimrisch', 'ScottDesJarlaisTN04', 'RepLarryBucshon', 'repchuck', 'RepWalberg', 'senbennetco', 'RepLoisFrankel', 'reptomrice', 'CongressmanClayHiggins', 'SenatorJohnHoeven', 'RepBonnie', 'RepHuizenga', 'CongressmanMikeDRogers', 'RepJoyceBeatty', 'SenatorSasse', 'CongressmanMarcVeasey', 'DianaDeGette', 'chrismurphyct', 'gregorymeeksny05', 'CongressmanDKDavis', 'CongresswomanRosaDeLauro', 'senatorbencardin', 'SenatorMikeRounds', 'RepAustinScott', 'mcmorrisrodgers', '152569121550', 'RepJuliaBrownley', 'USSenTinaSmith', 'repmarkpocan', 'timryan', 'herrerabeutler', 'sen.johncornyn', 'repstephenlynch', 'RepJoshG', 'RepJuanVargas', 'CongressmanGuthrie', 'repfrankpallone', '81058818750', 'tomcarper', 'repgosar', 'senjoniernst', 'CongressmanBillFoster', 'repcardenas', 'SenatorKaine', 'RepJimMcGovern', 'RepConorLamb', 'RepAdamSmith', 'RepDavidKustoff', 'RepDebbieLesko', 'RepKathleenRice', 'johnbarrasso', 'SenJackReed', 'RepJimmyPanetta', 'RepSteveChabot', 'reprobinkelly', 'pascrell', 'CongressmanKevinCramer', 'CongressmanGarretGraves', 'JohnKennedyLouisiana', 'senatorfeinstein', 'RepMullin', 'RepRubenGallego', 'RepScottPeters', 'joecourtney', 'RepSmucker', 'TomColeOK04', 'CongresswomanLindaSanchez', 'CongressmanDougLamborn', 'RepMikeJohnson', 'SenatorRichardBurr', 'repbrianmast', 'repjimjordan', 'RepMikeTurner', 'DonaldPayneJr', 'SenatorPatrickLeahy', 'RepTomReed', 'CongressmanBoyle', 'SenatorCortezMasto', 'ChelliePingree', 'RepRoKhanna', 'senatormikelee', 'CongressmanHalRogers', 'SenatorMarcoRubio', 'jerrymoran', 'chrisvanhollen', 'RepDelBene', 'CongressmanJimHimes', 'RepRobWittman', 'CongressmanRickAllen', 'jiminhofe', 'RepEspaillat', 'RepDebbieDingell', 'RepBradWenstrup', 'RepKarenBass', 'TXRandy14', 'RepFredUpton', 'SenatorTedCruz', 'RepresentativeMarcyKaptur', 'RepMikeGallagher', 'RepDonYoung', '395759603917487', 'susancollins', 'RepBillJohnson', '118514606128', 'RepRonEstes', 'RepSeanMaloney', 'RepMcKinley', 'SenatorAngusSKingJr', 'senronjohnson', 'repdavidschweikert', 'RepMikeThompson', 'RepMoolenaar', 'RepAndyBiggs', 'RepStephMurphy', 'sanfordbishop', 'RepSarbanes', 'CongressmanGaryPalmer', 'RepSteveWomack', 'RepLeeZeldin', 'RepDanKildee', 'JoeWilson', '81125319109', 'congressmancomer', '96007744606', 'GusBilirakis', '115356957005', 'RepDwightEvans', 'RepPerlmutter', 'RepJackieWalorski', 'CongressmanBradSchneider', 'CongressmanFredKeller', 'aumuaamata', 'CongressmanJodyHice', 'reprichardneal', 'replahood', 'SenatorDurbin', 'RepGrothman', 'RepLouCorrea', 'SenatorWhitehouse', 'RepLaMalfa', 'CongressmanMcHenry', 'RepVirginiaFoxx', 'CongressmanGerryConnolly', 'SenDanSullivan', 'RepAndyBarr', 'repbrianfitzpatrick', 'mdiazbalart', 'SenatorBlunt', 'CongresswomanTitus', 'stevenpalazzo', 'USSenatorLindseyGraham', 'RepDrewFerguson', 'mikecrapo', 'SenatorStabenow', 'DutchRupp', 'lloyddoggett', 'CongressmanEricSwalwell', 'judgecarter', 'RepJimCosta', 'RepAnnaEshoo', 'RepJackyRosen', 'CongressmanRaulRuizMD', 'senatorhirono', 'RepThomasMassie', 'congresswomanbonamici', 'SenLisaMurkowski', 'RepWilson', 'RepNormaTorres', 'RepJohnKatko', 'RepGraceNapolitano', 'JohnBoozman', 'RepHakeemJeffries', 'RepJudyChu', 'RepDaveJoyce', 'MartinHeinrich', '191056827594903', 'reploudermilk', 'CongressmanMattGaetz', 'jefffortenberry', 'RepRickLarsen', 'SenBlumenthal', 'RepAnnWagner', 'michaeltmccaul', 'RepRodneyDavis', 'RichardShelby', 'RepChrisSmith', 'doris.matsui', 'RepRutherfordFL', 'repsaludcarbajal', 'PeterWelch', 'CongressmanAndreCarson', 'NancyPelosi', 'SenDuckworth', 'CongressmanGT', 'congressmanraja', 'CongresswomanNorton', 'senatormenendez', 'JimCooper', 'MaxineWaters', 'SteveDainesMT', 'USCongressmanVicenteGonzalez', 'RepCarolynMaloney', 'CongressmanDarrenSoto', 'senschumer', 'repstaceyplaskett', 'CongresswomanAdams', 'Rep.Billy.Long', 'congressmanbuddycarter', 'SenatorShaheen', 'USRepKathyCastor', 'boblatta', 'CongresswomanSinema', 'RepJennifferGonzalezColon', 'repbettymccollum', 'repyvettedclarke', 'RepCharlieCrist', 'usrepmikedoyle', 'senatortoomey', 'RepJayapal', '326420614138023', 'RepLowenthal', 'SenatorSherrodBrown', 'jerrymcnerney', 'repjasonsmith', 'CongresswomanEBJtx30', 'CongresswomanSheilaJacksonLee', '70063393423', '63158229861', 'RepWebster', '214258646163', 'JackieSpeier', 'kevinbrady', 'senrobportman', 'Rep.BluntRochester', 'RepBrianBabin', 'repraskin', 'reptomemmer', 'repdanbishop', 'repmikequigley', 'MarkRWarner', 'EdJMarkey', 'senatorsanders', 'repohalleran', 'grassley', 'RepRalphNorman', 'RepDWS', 'RepMorganGriffith', 'MarkAmodeiNV2', 'RepJimmyGomez', 'Rep.Grijalva', 'repronkind', 'janschakowsky', 'SenKirstenGillibrand', 'repkenbuck', 'reppeteaguilar', 'RepDavidEPrice', 'SenatorToddYoung', 'CongressmanJimLangevin', 'RepFrenchHill', 'CongresswomanClark', 'SenatorBobCasey', 'Congresswoman.Hartzler', 'derek.kilmer', 'reptrentkelly', 'SenMarkKelly', 'congressmanbobbyrush', 'CongresswomanBarragan', 'RepKayGranger', 'michaelcburgess', 'RepPeterDeFazio', 'SenatorWicker', 'RepMikeGarcia', 'RepAlLawsonJr', 'RepNewhouse', '8037068318']

set_of_pol_pgnames = {'SenatorThomTillis', 'USCongressmanFilemonVela', 'reppaultonko', 'DonaldNorcrossNJ', '153423912663', 'repalgreen', 'RepDonBeyer', 'RepRogerWilliams', 'CongressmanSethMoulton', 'replizcheney', 'RepRoybalAllard', 'mitchmcconnell', 'SenatorRandPaul', 'RepSteveScalise', 'RepMoBrooks', 'CongressmanAlexMooney', 'RepJackBergman', 'RepWesterman', 'RepMcEachin', 'CongresswomanAnnieKuster', 'RepFrankLucas', 'CongressmanTedDeutch', 'GwenSMoore', 'RepTedLieu', 'RepGregMurphy', 'senatordebfischer', 'RepBobGibbs', 'AndyHarrisMD', 'WhipHoyer', 'CongressmanMichaelF.Q.SanNicolas', 'congressmangkbutterfield', 'SenatorTimScott', 'RepJohnCurtis', 'CongressmanNadler', '50375006903', 'RepBarbaraLee', 'jeffmerkley', 'JoeManchinIII', 'RepBost', 'CongressmanBuchanan', 'RepJeffDuncan', 'Congressman.Keating', 'RepSewell', 'RepRickCrawford', 'RepChrisStewart', 'zoelofgren', 'CongressmanWarrenDavidson', 'drnealdunnfl2', 'RepTomSuozzi', 'SenatorLankford', 'RepCheri', 'RepKinzinger', 'marshablackburn', 'SenBrianSchatz', 'RepDonBacon', 'SenatorHassan', 'senshelley', 'senatorcantwell', 'BlaineLuetkemeyer', '113303673339', 'RepJohnLarson', 'senatortester', 'RepRouzer', 'RobertAderholt', 'reptrey', 'jameseclyburn', 'RepRichHudson', 'RickScottSenOffice', 'SenGaryPeters', 'repschrader', 'RepKweisiMfume', 'CongressmanSteveCohen', 'senatortammybaldwin', 'RepAdamSchiff', 'RepBobbyScott', 'RepEliseStefanik', 'senatorelizabethwarren', 'CongressmanMattCartwright', 'RepAmiBera', 'CongressmanKevinMcCarthy', 'RepAnthonyBrown', 'repgaramendi', 'CongressmanDavidCicilline', 'RepJimBanks', 'JodeyArrington', 'repgracemeng', 'bill.posey15', 'senatorchriscoons', 'RepTedBudd', 'RepBrianHiggins', 'RepMarkTakano', 'CongressmanBennieGThompson', 'emanuelcleaverii', 'RepMarkDeSaulnier', 'RepHuffman', 'SenatorTomCotton', 'repscottperry', 'RepresentativeValDemings', 'senatorjimrisch', 'ScottDesJarlaisTN04', 'RepLarryBucshon', 'repchuck', 'RepWalberg', 'senbennetco', 'RepLoisFrankel', 'reptomrice', 'CongressmanClayHiggins', 'SenatorJohnHoeven', 'RepBonnie', 'RepHuizenga', 'CongressmanMikeDRogers', 'RepJoyceBeatty', 'SenatorSasse', 'CongressmanMarcVeasey', 'DianaDeGette', 'chrismurphyct', 'gregorymeeksny05', 'CongressmanDKDavis', 'CongresswomanRosaDeLauro', 'senatorbencardin', 'SenatorMikeRounds', 'RepAustinScott', 'mcmorrisrodgers', '152569121550', 'RepJuliaBrownley', 'USSenTinaSmith', 'repmarkpocan', 'timryan', 'herrerabeutler', 'sen.johncornyn', 'repstephenlynch', 'RepJoshG', 'RepJuanVargas', 'CongressmanGuthrie', 'repfrankpallone', '81058818750', 'tomcarper', 'repgosar', 'senjoniernst', 'CongressmanBillFoster', 'repcardenas', 'SenatorKaine', 'RepJimMcGovern', 'RepConorLamb', 'RepAdamSmith', 'RepDavidKustoff', 'RepDebbieLesko', 'RepKathleenRice', 'johnbarrasso', 'SenJackReed', 'RepJimmyPanetta', 'RepSteveChabot', 'reprobinkelly', 'pascrell', 'CongressmanKevinCramer', 'CongressmanGarretGraves', 'JohnKennedyLouisiana', 'senatorfeinstein', 'RepMullin', 'RepRubenGallego', 'RepScottPeters', 'joecourtney', 'RepSmucker', 'TomColeOK04', 'CongresswomanLindaSanchez', 'CongressmanDougLamborn', 'RepMikeJohnson', 'SenatorRichardBurr', 'repbrianmast', 'repjimjordan', 'RepMikeTurner', 'DonaldPayneJr', 'SenatorPatrickLeahy', 'RepTomReed', 'CongressmanBoyle', 'SenatorCortezMasto', 'ChelliePingree', 'RepRoKhanna', 'senatormikelee', 'CongressmanHalRogers', 'SenatorMarcoRubio', 'jerrymoran', 'chrisvanhollen', 'RepDelBene', 'CongressmanJimHimes', 'RepRobWittman', 'CongressmanRickAllen', 'jiminhofe', 'RepEspaillat', 'RepDebbieDingell', 'RepBradWenstrup', 'RepKarenBass', 'TXRandy14', 'RepFredUpton', 'SenatorTedCruz', 'RepresentativeMarcyKaptur', 'RepMikeGallagher', 'RepDonYoung', '395759603917487', 'susancollins', 'RepBillJohnson', '118514606128', 'RepRonEstes', 'RepSeanMaloney', 'RepMcKinley', 'SenatorAngusSKingJr', 'senronjohnson', 'repdavidschweikert', 'RepMikeThompson', 'RepMoolenaar', 'RepAndyBiggs', 'RepStephMurphy', 'sanfordbishop', 'RepSarbanes', 'CongressmanGaryPalmer', 'RepSteveWomack', 'RepLeeZeldin', 'RepDanKildee', 'JoeWilson', '81125319109', 'congressmancomer', '96007744606', 'GusBilirakis', '115356957005', 'RepDwightEvans', 'RepPerlmutter', 'RepJackieWalorski', 'CongressmanBradSchneider', 'CongressmanFredKeller', 'aumuaamata', 'CongressmanJodyHice', 'reprichardneal', 'replahood', 'SenatorDurbin', 'RepGrothman', 'RepLouCorrea', 'SenatorWhitehouse', 'RepLaMalfa', 'CongressmanMcHenry', 'RepVirginiaFoxx', 'CongressmanGerryConnolly', 'SenDanSullivan', 'RepAndyBarr', 'repbrianfitzpatrick', 'mdiazbalart', 'SenatorBlunt', 'CongresswomanTitus', 'stevenpalazzo', 'USSenatorLindseyGraham', 'RepDrewFerguson', 'mikecrapo', 'SenatorStabenow', 'DutchRupp', 'lloyddoggett', 'CongressmanEricSwalwell', 'judgecarter', 'RepJimCosta', 'RepAnnaEshoo', 'RepJackyRosen', 'CongressmanRaulRuizMD', 'senatorhirono', 'RepThomasMassie', 'congresswomanbonamici', 'SenLisaMurkowski', 'RepWilson', 'RepNormaTorres', 'RepJohnKatko', 'RepGraceNapolitano', 'JohnBoozman', 'RepHakeemJeffries', 'RepJudyChu', 'RepDaveJoyce', 'MartinHeinrich', '191056827594903', 'reploudermilk', 'CongressmanMattGaetz', 'jefffortenberry', 'RepRickLarsen', 'SenBlumenthal', 'RepAnnWagner', 'michaeltmccaul', 'RepRodneyDavis', 'RichardShelby', 'RepChrisSmith', 'doris.matsui', 'RepRutherfordFL', 'repsaludcarbajal', 'PeterWelch', 'CongressmanAndreCarson', 'NancyPelosi', 'SenDuckworth', 'CongressmanGT', 'congressmanraja', 'CongresswomanNorton', 'senatormenendez', 'JimCooper', 'MaxineWaters', 'SteveDainesMT', 'USCongressmanVicenteGonzalez', 'RepCarolynMaloney', 'CongressmanDarrenSoto', 'senschumer', 'repstaceyplaskett', 'CongresswomanAdams', 'Rep.Billy.Long', 'congressmanbuddycarter', 'SenatorShaheen', 'USRepKathyCastor', 'boblatta', 'CongresswomanSinema', 'RepJennifferGonzalezColon', 'repbettymccollum', 'repyvettedclarke', 'RepCharlieCrist', 'usrepmikedoyle', 'senatortoomey', 'RepJayapal', '326420614138023', 'RepLowenthal', 'SenatorSherrodBrown', 'jerrymcnerney', 'repjasonsmith', 'CongresswomanEBJtx30', 'CongresswomanSheilaJacksonLee', '70063393423', '63158229861', 'RepWebster', '214258646163', 'JackieSpeier', 'kevinbrady', 'senrobportman', 'Rep.BluntRochester', 'RepBrianBabin', 'repraskin', 'reptomemmer', 'repdanbishop', 'repmikequigley', 'MarkRWarner', 'EdJMarkey', 'senatorsanders', 'repohalleran', 'grassley', 'RepRalphNorman', 'RepDWS', 'RepMorganGriffith', 'MarkAmodeiNV2', 'RepJimmyGomez', 'Rep.Grijalva', 'repronkind', 'janschakowsky', 'SenKirstenGillibrand', 'repkenbuck', 'reppeteaguilar', 'RepDavidEPrice', 'SenatorToddYoung', 'CongressmanJimLangevin', 'RepFrenchHill', 'CongresswomanClark', 'SenatorBobCasey', 'Congresswoman.Hartzler', 'derek.kilmer', 'reptrentkelly', 'SenMarkKelly', 'congressmanbobbyrush', 'CongresswomanBarragan', 'RepKayGranger', 'michaelcburgess', 'RepPeterDeFazio', 'SenatorWicker', 'RepMikeGarcia', 'RepAlLawsonJr', 'RepNewhouse', '8037068318'}

polnames_already_fetched = {'Senator Thom Tillis', 'Congressman Filemon Vela', 'Paul D. Tonko', 'Donald Norcross', 'Gregorio Kilili Camacho Sablan', 'Rep. Al Green', 'Congressman Don Beyer', 'Roger Williams', 'Congressman Seth Moulton','Rep. Liz Cheney', 'Gwen S. Moore'}


new = set_of_pol_pgnames.difference(polnames_already_fetched)

{'113303673339',
 '115356957005',
 '118514606128',
 '152569121550',
 '153423912663',
 '191056827594903',
 '214258646163',
 '326420614138023',
 '395759603917487',
 '50375006903',
 '63158229861',
 '70063393423',
 '8037068318',
 '81058818750',
 '81125319109',
 '96007744606',
 'AndyHarrisMD',
 'BlaineLuetkemeyer',
 'ChelliePingree',
 'Congressman.Keating',
 'CongressmanAlexMooney',
 'CongressmanAndreCarson',
 'CongressmanBennieGThompson',
 'CongressmanBillFoster',
 'CongressmanBoyle',
 'CongressmanBradSchneider',
 'CongressmanBuchanan',
 'CongressmanClayHiggins',
 'CongressmanDKDavis',
 'CongressmanDarrenSoto',
 'CongressmanDavidCicilline',
 'CongressmanDougLamborn',
 'CongressmanEricSwalwell',
 'CongressmanFredKeller',
 'CongressmanGT',
 'CongressmanGarretGraves',
 'CongressmanGaryPalmer',
 'CongressmanGerryConnolly',
 'CongressmanGuthrie',
 'CongressmanHalRogers',
 'CongressmanJimHimes',
 'CongressmanJimLangevin',
 'CongressmanJodyHice',
 'CongressmanKevinCramer',
 'CongressmanKevinMcCar

In [6]:
list_pol_fbpages = []
list_of_failed_request = []
fb_pop_up_blocked = []
nb_failed_requests = 0
nb_popup_blocked = 0

for politician in list_of_pol_pgnames:
    
    request  = 'https://www.facebook.com/' + politician
    
    #priting request just to ensure
    #print(request)

    r = requests.get(request)
    
    if r.status_code == 200:
        
        pg_content = str(r.content)

        look_up_word ='og:title" content="'

        a = pg_content.split(look_up_word)
        name_split = re.split('"', a[-1], maxsplit=1)
        
        pol_fb_page_name = name_split[0]

        if (pol_fb_page_name == "b'<!DOCTYPE html>\n<html lang=") | (pol_fb_page_name == "Log p&#xe5;, eller opret en profil for at se indholdet"):
            fb_pop_up_blocked.append(politician)
            nb_popup_blocked += 1
        else:
            list_pol_fbpages.append(pol_fb_page_name)

        #printing it as well as a safety if shit goes south as crashes.
        print(pol_fb_page_name)

        #Figure out how long sleep should be to not get "siden findes ikke / page not found"
        #don't think 30 secs will be sufficient however...
        time.sleep(randint(2,6))
    else:
        list_of_failed_request.append(politician)
        nb_failed_requests += 1

#we need to figure out how to log-in with the requests - so we don't get this log-on to facebook redirection banner instead of the page of the politician // sometimes the 

b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=
b'<!DOCTYPE html>\n<html lang=


KeyboardInterrupt: 

In [7]:
list_pol_fbpages

#Results fetched so far
set_of_polnames_already_fetched = {'Senator Thom Tillis', 'Congressman Filemon Vela', 'Paul D. Tonko', 'Donald Norcross', 'Gregorio Kilili Camacho Sablan', 'Rep. Al Green', 'Congressman Don Beyer', 'Roger Williams', 'Congressman Seth Moulton','Rep. Liz Cheney', 'Gwen S. Moore'}


#To create new list to fetch
